In [ ]:
# !pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import joblib
import pickle
import json

# ml frameworks
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr


from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
#time management
from tqdm import tqdm

MultiClassification

In [ ]:
X_train_csv = pd.read_csv("/content/drive/MyDrive/GR-III/CARVEME/50/X_train_GR-III.csv")
X_val_csv = pd.read_csv("/content/drive/MyDrive/GR-III/CARVEME/50/X_val_GR-III.csv")
X_test_csv = pd.read_csv("/content/drive/MyDrive/GR-III/CARVEME/50/X_test_GR-III.csv")
y_train_csv = pd.read_csv("/content/drive/MyDrive/GR-III/CARVEME/50/y_train_GR-III.csv")
y_val_csv = pd.read_csv("/content/drive/MyDrive/GR-III/CARVEME/50/y_val_GR-III.csv")
y_test_csv = pd.read_csv("/content/drive/MyDrive/GR-III/CARVEME/50/y_test_GR-III.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv):
    X_train = X_train_csv.to_numpy()
    X_val = X_val_csv.to_numpy()
    X_test = X_test_csv.to_numpy()
    y_train = y_train_csv.to_numpy()
    y_val = y_val_csv.to_numpy()
    y_test = y_test_csv.to_numpy()

    # Save the arrays as .npy files
    np.save("X_train.npy", X_train)
    np.save("X_val.npy", X_val)
    np.save("X_test.npy", X_test)
    np.save("y_train.npy", y_train)
    np.save("y_val.npy", y_val)
    np.save("y_test.npy", y_test)

    print("CSV onverted to .npy format.")


In [ ]:
to_npy(X_train_csv, X_val_csv, X_test_csv, y_train_csv, y_val_csv, y_test_csv)

CSV onverted to .npy format.


In [ ]:
train_list = ["X_train.npy", "X_val.npy", "X_test.npy", "y_train.npy", "y_val.npy", "y_test.npy"]

In [ ]:
def load(train_list):
    X_train = np.load(train_list[0])
    X_val = np.load(train_list[1])
    X_test = np.load(train_list[2])
    y_train = np.load(train_list[3])
    y_val = np.load(train_list[4])
    y_test = np.load(train_list[5])
    return X_train, X_val, X_test, y_train, y_val, y_test


def scatter_true_vs_pred(y_true, y_pred):
    """
    Description: Scatter plot of true vs. predicted values.
    Arguments: y_true (array-like): Ground truth values;
               y_pred (array-like): Predicted values.
    Outputs: Sc.
    """
    plt.scatter(y_true, y_pred, alpha=0.5)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], linestyle="--", color="red")
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title("True vs. Predicted Values")
    plt.show()


def regression_metrics(y_true, y_pred):
    """
    Description: Calculate various regression metrics.
    Arguments: y_true (array-like): Ground truth values;
               y_pred (array-like): Predicted values.
    Outputs:
        dict: A dictionary containing MAE, MSE, RMSE, R².
    """
    metrics = {
        "MAE": mean_absolute_error(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_true, y_pred)),
        "R² Score": r2_score(y_true, y_pred),
        "Pearson Correlation": pearsonr(y_true, y_pred)[0] if len(y_true) > 1 else np.nan
    }
    return metrics


def plot_residuals(y_true, y_pred):
    """
    Description: Plot residuals (errors).
    Arguments: y_true (array-like): Ground truth values;
               y_pred (array-like): Predicted values.
    Outputs: Residuals plot.
    """
    residuals = y_true - y_pred
    plt.scatter(y_pred, residuals, alpha=0.5)
    plt.axhline(0, linestyle="--", color="red")
    plt.xlabel("Predicted Values")
    plt.ylabel("Residuals (True - Predicted)")
    plt.title("Residuals Plot")
    plt.show()



def train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed):
    """
    Description: Trains and evaluates multiple regression models using train, validation, and test splits.
                 Saves the results, regression metrics, and scatter/residual plots into a corresponding directory.

    Arguments:
        models (dict): Dictionary of regression models;
        X_train (ndarray): Training feature data;
        y_train (ndarray): Training target data;
        X_val (ndarray): Validation feature data;
        y_val (ndarray): Validation target data;
        X_test (ndarray): Test feature data;
        y_test (ndarray): Test target data;
        seed (int): Random seed for reproducibility.

    Outputs:
        A dictionary containing validation and test metrics for each model.
    """

    results = {}

    for model_name, model in tqdm(models.items(), desc="Training Models", unit="model"):
        print(f"\nTraining and evaluating: {model_name}")

        # Create a storage directory for results
        model_dir = f"results/{model_name.replace(' ', '_')}"
        if os.path.exists(model_dir):
            shutil.rmtree(model_dir)  # Remove
        os.makedirs(model_dir)

        # Train the model
        model.fit(X_train, y_train)

        # Save the trained model
        model_path = os.path.join(model_dir, "model.pkl")
        joblib.dump(model, model_path)
        print(f"Model saved at {model_path}")

        # Validation
        y_val_pred = model.predict(X_val)
        val_r2 = r2_score(y_val, y_val_pred)
        val_mae = mean_absolute_error(y_val, y_val_pred)
        val_mse = mean_squared_error(y_val, y_val_pred)

        # Final test
        y_test_pred = model.predict(X_test)
        test_r2 = r2_score(y_test, y_test_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)

        # Store results
        results[model_name] = {
            "Validation R²": val_r2,
            "Validation MAE": val_mae,
            "Validation MSE": val_mse,
            "Test R²": test_r2,
            "Test MAE": test_mae,
            "Test MSE": test_mse,
        }

        print(f"Validation Report for {model_name}:")
        print(f"R²: {val_r2:.4f}, MAE: {val_mae:.4f}, MSE: {val_mse:.4f}")

        # # Scatter plot for True vs Predicted
        # plt.figure(figsize=(6, 6))
        # plt.scatter(y_val, y_val_pred, alpha=0.5)
        # plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], linestyle="--", color="red")
        # plt.xlabel("True Values")
        # plt.ylabel("Predicted Values")
        # plt.title(f"True vs Predicted - {model_name}")
        # plt.tight_layout()
        # plt.savefig(os.path.join(model_dir, "true_vs_predicted.png"))
        # plt.close()

        # # Residual plot (True - Predicted)
        # residuals = y_val - y_val_pred
        # plt.figure(figsize=(6, 6))
        # plt.scatter(y_val_pred, residuals, alpha=0.5)
        # plt.axhline(0, linestyle="--", color="red")
        # plt.xlabel("Predicted Values")
        # plt.ylabel("Residuals (True - Predicted)")
        # plt.title(f"Residuals Plot - {model_name}")
        # plt.tight_layout()
        # plt.savefig(os.path.join(model_dir, "residuals_plot.png"))
        # plt.close()

    # Recap of all models' results
    print("\nSummary of Validation and Test Metrics:")
    for model_name, metrics in results.items():
        print(
            f"{model_name}: "
            f"Val R² = {metrics['Validation R²']:.4f}, "
            f"Val MAE = {metrics['Validation MAE']:.4f}, "
            f"Val MSE = {metrics['Validation MSE']:.4f}, "
            f"Test R² = {metrics['Test R²']:.4f}, "
            f"Test MAE = {metrics['Test MAE']:.4f}, "
            f"Test MSE = {metrics['Test MSE']:.4f}"
        )

    # Save all model
    all_results_path = "results/all_model_results.json"
    with open(all_results_path, "w") as f:
        json.dump(results, f, indent=4)
    print(f"\nAll model results saved to {all_results_path}")

    return results

#AGORA-GR-I-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.074700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 828
[LightGBM] [Info] Number of data points in the train set: 63265, number of used features: 414
[LightGBM] [Info] Start training from score 85.022476
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:02<00:04,  2.13s/model]

Validation Report for LightGBM:
R²: 0.6092, MAE: 13.5600, MSE: 343.2035

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:47:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:04<00:02,  2.18s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099555 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 828
[LightGBM] [Info] Number of data points in the train set: 63265, number of used features: 414
[LightGBM] [Info] Start training from score 85.022476
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:07<00:00,  2.37s/model]

Validation Report for CatBoost:
R²: 0.6092, MAE: 13.5600, MSE: 343.2035

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.6092, Val MAE = 13.5600, Val MSE = 343.2035, Test R² = 0.6042, Test MAE = 13.7143, Test MSE = 350.5980
XGBoost: Val R² = 0.6405, Val MAE = 12.6578, Val MSE = 315.7621, Test R² = 0.6335, Test MAE = 12.8072, Test MSE = 324.6611
CatBoost: Val R² = 0.6092, Val MAE = 13.5600, Val MSE = 343.2035, Test R² = 0.6042, Test MAE = 13.7143, Test MSE = 350.5980

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.6092268703701995,
  'Validation MAE': 13.559978437186842,
  'Validation MSE': 343.2034600258781,
  'Test R²': 0.6041945279961034,
  'Test MAE': 13.714320288372694,
  'Test MSE': 350.59803111578395},
 'XGBoost': {'Validation R²': 0.640471729189166,
  'Validation MAE': 12.657780989020825,
  'Validation MSE': 315.7621063564277,
  'Test R²': 0.6334758774368903,
  'Test MAE': 12.807248398308763,
  'Test MSE': 324.6610893893895},
 'CatBoost': {'Validation R²': 0.6092268703701995,
  'Validation MAE': 13.559978437186842,
  'Validation MSE': 343.2034600258781,
  'Test R²': 0.6041945279961034,
  'Test MAE': 13.714320288372694,
  'Test MSE': 350.59803111578395}}

# AGORA-GR-II-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.153423 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 828
[LightGBM] [Info] Number of data points in the train set: 132532, number of used features: 414
[LightGBM] [Info] Start training from score 80.634781
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:04<00:08,  4.14s/model]

Validation Report for LightGBM:
R²: 0.5167, MAE: 14.5490, MSE: 370.4922

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:49:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:13<00:07,  7.45s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.158932 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 828
[LightGBM] [Info] Number of data points in the train set: 132532, number of used features: 414
[LightGBM] [Info] Start training from score 80.634781
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:18<00:00,  6.04s/model]

Validation Report for CatBoost:
R²: 0.5167, MAE: 14.5490, MSE: 370.4922

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.5167, Val MAE = 14.5490, Val MSE = 370.4922, Test R² = 0.5230, Test MAE = 14.5867, Test MSE = 372.9865
XGBoost: Val R² = 0.5607, Val MAE = 13.6300, Val MSE = 336.7745, Test R² = 0.5671, Test MAE = 13.6396, Test MSE = 338.5262
CatBoost: Val R² = 0.5167, Val MAE = 14.5490, Val MSE = 370.4922, Test R² = 0.5230, Test MAE = 14.5867, Test MSE = 372.9865

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.5166871079061213,
  'Validation MAE': 14.54895661138813,
  'Validation MSE': 370.492222778473,
  'Test R²': 0.5230071795093698,
  'Test MAE': 14.586708335490306,
  'Test MSE': 372.98645690447796},
 'XGBoost': {'Validation R²': 0.5606724473436165,
  'Validation MAE': 13.629974278172305,
  'Validation MSE': 336.7744667565668,
  'Test R²': 0.5670765997858762,
  'Test MAE': 13.639604934311174,
  'Test MSE': 338.5261962450801},
 'CatBoost': {'Validation R²': 0.5166871079061213,
  'Validation MAE': 14.54895661138813,
  'Validation MSE': 370.492222778473,
  'Test R²': 0.5230071795093698,
  'Test MAE': 14.586708335490306,
  'Test MSE': 372.98645690447796}}

#AGORA-GR-III-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.266318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 830
[LightGBM] [Info] Number of data points in the train set: 242313, number of used features: 415
[LightGBM] [Info] Start training from score 90.990516
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:07<00:15,  7.75s/model]

Validation Report for LightGBM:
R²: 0.5987, MAE: 14.7367, MSE: 369.9508

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:51:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.270576 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 830
[LightGBM] [Info] Number of data points in the train set: 242313, number of used features: 415
[LightGBM] [Info] Start training from score 90.990516
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:26<00:00,  8.89s/model]

Validation Report for CatBoost:
R²: 0.5987, MAE: 14.7367, MSE: 369.9508

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.5987, Val MAE = 14.7367, Val MSE = 369.9508, Test R² = 0.5943, Test MAE = 14.7598, Test MSE = 372.7542
XGBoost: Val R² = 0.6368, Val MAE = 13.8411, Val MSE = 334.8362, Test R² = 0.6326, Test MAE = 13.8384, Test MSE = 337.5833
CatBoost: Val R² = 0.5987, Val MAE = 14.7367, Val MSE = 369.9508, Test R² = 0.5943, Test MAE = 14.7598, Test MSE = 372.7542

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.5987373101312441,
  'Validation MAE': 14.73674558959337,
  'Validation MSE': 369.9507969148309,
  'Test R²': 0.5942692188400018,
  'Test MAE': 14.759778940722446,
  'Test MSE': 372.75423000786674},
 'XGBoost': {'Validation R²': 0.636823886347573,
  'Validation MAE': 13.84107421446232,
  'Validation MSE': 334.83624582712093,
  'Test R²': 0.6325516753925461,
  'Test MAE': 13.838376274052907,
  'Test MSE': 337.58325388854206},
 'CatBoost': {'Validation R²': 0.5987373101312441,
  'Validation MAE': 14.73674558959337,
  'Validation MSE': 369.9507969148309,
  'Test R²': 0.5942692188400018,
  'Test MAE': 14.759778940722446,
  'Test MSE': 372.75423000786674}}

#AGORA-GR-I-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017408 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 688
[LightGBM] [Info] Number of data points in the train set: 10048, number of used features: 344
[LightGBM] [Info] Start training from score 75.777139


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:00<00:00,  2.37model/s]

Model saved at results/LightGBM/model.pkl
Validation Report for LightGBM:
R²: 0.4220, MAE: 15.2557, MSE: 410.0791

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:55:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:02<00:01,  1.47s/model]

Model saved at results/XGBoost/model.pkl
Validation Report for XGBoost:
R²: 0.3682, MAE: 16.0105, MSE: 448.2937

Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021674 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 688
[LightGBM] [Info] Number of data points in the train set: 10048, number of used features: 344
[LightGBM] [Info] Start training from score 75.777139


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:03<00:00,  1.02s/model]

Model saved at results/CatBoost/model.pkl
Validation Report for CatBoost:
R²: 0.4220, MAE: 15.2557, MSE: 410.0791

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.4220, Val MAE = 15.2557, Val MSE = 410.0791, Test R² = 0.4101, Test MAE = 15.1051, Test MSE = 402.8497
XGBoost: Val R² = 0.3682, Val MAE = 16.0105, Val MSE = 448.2937, Test R² = 0.3527, Test MAE = 15.8312, Test MSE = 442.0807
CatBoost: Val R² = 0.4220, Val MAE = 15.2557, Val MSE = 410.0791, Test R² = 0.4101, Test MAE = 15.1051, Test MSE = 402.8497

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.4220474645985287,
  'Validation MAE': 15.255739943709271,
  'Validation MSE': 410.0790869263016,
  'Test R²': 0.41010976664073473,
  'Test MAE': 15.105055916876674,
  'Test MSE': 402.8496839231884},
 'XGBoost': {'Validation R²': 0.3681889610848823,
  'Validation MAE': 16.010499958722257,
  'Validation MSE': 448.2937232350618,
  'Test R²': 0.35266406396426087,
  'Test MAE': 15.831191578038958,
  'Test MSE': 442.0806829417272},
 'CatBoost': {'Validation R²': 0.4220474645985287,
  'Validation MAE': 15.255739943709271,
  'Validation MSE': 410.0790869263016,
  'Test R²': 0.41010976664073473,
  'Test MAE': 15.105055916876674,
  'Test MSE': 402.8496839231884}}

# AGORA-GR-II-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.067895 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 774
[LightGBM] [Info] Number of data points in the train set: 43653, number of used features: 387
[LightGBM] [Info] Start training from score 63.244210


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:01<00:02,  1.27s/model]

Model saved at results/LightGBM/model.pkl
Validation Report for LightGBM:
R²: 0.3489, MAE: 14.5974, MSE: 365.4987

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:54:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:02<00:01,  1.42s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068028 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 774
[LightGBM] [Info] Number of data points in the train set: 43653, number of used features: 387
[LightGBM] [Info] Start training from score 63.244210


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:04<00:00,  1.33s/model]

Model saved at results/CatBoost/model.pkl
Validation Report for CatBoost:
R²: 0.3489, MAE: 14.5974, MSE: 365.4987

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.3489, Val MAE = 14.5974, Val MSE = 365.4987, Test R² = 0.3560, Test MAE = 14.5384, Test MSE = 360.9291
XGBoost: Val R² = 0.3433, Val MAE = 14.5771, Val MSE = 368.6329, Test R² = 0.3466, Test MAE = 14.5864, Test MSE = 366.1493
CatBoost: Val R² = 0.3489, Val MAE = 14.5974, Val MSE = 365.4987, Test R² = 0.3560, Test MAE = 14.5384, Test MSE = 360.9291

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.3489170577885986,
  'Validation MAE': 14.597368710235354,
  'Validation MSE': 365.4987060522398,
  'Test R²': 0.3559531638164911,
  'Test MAE': 14.538440497290292,
  'Test MSE': 360.9291268089409},
 'XGBoost': {'Validation R²': 0.34333398430339657,
  'Validation MAE': 14.577053353306834,
  'Validation MSE': 368.6328783709078,
  'Test R²': 0.3466382190202073,
  'Test MAE': 14.586431448314205,
  'Test MSE': 366.14929823547703},
 'CatBoost': {'Validation R²': 0.3489170577885986,
  'Validation MAE': 14.597368710235354,
  'Validation MSE': 365.4987060522398,
  'Test R²': 0.3559531638164911,
  'Test MAE': 14.538440497290292,
  'Test MSE': 360.9291268089409}}

#AGORA-GR-III-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.469093 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 830
[LightGBM] [Info] Number of data points in the train set: 284652, number of used features: 415
[LightGBM] [Info] Start training from score 71.914278
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:06<00:13,  6.53s/model]

Validation Report for LightGBM:
R²: 0.4544, MAE: 14.9849, MSE: 373.2057

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:53:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.473890 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 830
[LightGBM] [Info] Number of data points in the train set: 284652, number of used features: 415
[LightGBM] [Info] Start training from score 71.914278
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:22<00:00,  7.48s/model]

Validation Report for CatBoost:
R²: 0.4544, MAE: 14.9849, MSE: 373.2057

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.4544, Val MAE = 14.9849, Val MSE = 373.2057, Test R² = 0.4534, Test MAE = 14.9460, Test MSE = 372.1938
XGBoost: Val R² = 0.4681, Val MAE = 14.7358, Val MSE = 363.8316, Test R² = 0.4702, Test MAE = 14.6637, Test MSE = 360.7674
CatBoost: Val R² = 0.4544, Val MAE = 14.9849, Val MSE = 373.2057, Test R² = 0.4534, Test MAE = 14.9460, Test MSE = 372.1938

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.45435315111497154,
  'Validation MAE': 14.984922608227626,
  'Validation MSE': 373.2056751463514,
  'Test R²': 0.4534048400189986,
  'Test MAE': 14.945956231553746,
  'Test MSE': 372.19378591646074},
 'XGBoost': {'Validation R²': 0.46805861852319186,
  'Validation MAE': 14.735822942203278,
  'Validation MSE': 363.8315566524335,
  'Test R²': 0.4701853524006566,
  'Test MAE': 14.663721749763583,
  'Test MSE': 360.76740879090335},
 'CatBoost': {'Validation R²': 0.45435315111497154,
  'Validation MAE': 14.984922608227626,
  'Validation MSE': 373.2056751463514,
  'Test R²': 0.4534048400189986,
  'Test MAE': 14.945956231553746,
  'Test MSE': 372.19378591646074}}

#CARVEME-GR-I-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.262023 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 123373, number of used features: 489
[LightGBM] [Info] Start training from score 41.951965
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:04<00:09,  4.98s/model]

Validation Report for LightGBM:
R²: 0.3094, MAE: 9.0997, MSE: 132.0320

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:57:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:09<00:04,  4.74s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.204552 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 123373, number of used features: 489
[LightGBM] [Info] Start training from score 41.951965
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:13<00:00,  4.57s/model]

Validation Report for CatBoost:
R²: 0.3094, MAE: 9.0997, MSE: 132.0320

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.3094, Val MAE = 9.0997, Val MSE = 132.0320, Test R² = 0.3086, Test MAE = 9.0873, Test MSE = 131.6059
XGBoost: Val R² = 0.3852, Val MAE = 8.4459, Val MSE = 117.5423, Test R² = 0.3863, Test MAE = 8.4351, Test MSE = 116.8143
CatBoost: Val R² = 0.3094, Val MAE = 9.0997, Val MSE = 132.0320, Test R² = 0.3086, Test MAE = 9.0873, Test MSE = 131.6059

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.3093933509363136,
  'Validation MAE': 9.099730861721103,
  'Validation MSE': 132.03196022565163,
  'Test R²': 0.3085649923422942,
  'Test MAE': 9.087271347086409,
  'Test MSE': 131.6058862567178},
 'XGBoost': {'Validation R²': 0.38518283037750256,
  'Validation MAE': 8.445892044750428,
  'Validation MSE': 117.5423320868714,
  'Test R²': 0.38627753368242634,
  'Test MAE': 8.43514352089679,
  'Test MSE': 116.81428941383277},
 'CatBoost': {'Validation R²': 0.3093933509363136,
  'Validation MAE': 9.099730861721103,
  'Validation MSE': 132.03196022565163,
  'Test R²': 0.3085649923422942,
  'Test MAE': 9.087271347086409,
  'Test MSE': 131.6058862567178}}

#CARVEME-GR-II-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.230806 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 123373, number of used features: 489
[LightGBM] [Info] Start training from score 41.951965
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:04<00:09,  4.68s/model]

Validation Report for LightGBM:
R²: 0.3094, MAE: 9.0997, MSE: 132.0320

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:59:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:09<00:04,  4.62s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.199202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 123373, number of used features: 489
[LightGBM] [Info] Start training from score 41.951965
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:13<00:00,  4.47s/model]

Validation Report for CatBoost:
R²: 0.3094, MAE: 9.0997, MSE: 132.0320

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.3094, Val MAE = 9.0997, Val MSE = 132.0320, Test R² = 0.3086, Test MAE = 9.0873, Test MSE = 131.6059
XGBoost: Val R² = 0.3852, Val MAE = 8.4459, Val MSE = 117.5423, Test R² = 0.3863, Test MAE = 8.4351, Test MSE = 116.8143
CatBoost: Val R² = 0.3094, Val MAE = 9.0997, Val MSE = 132.0320, Test R² = 0.3086, Test MAE = 9.0873, Test MSE = 131.6059

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.3093933509363136,
  'Validation MAE': 9.099730861721103,
  'Validation MSE': 132.03196022565163,
  'Test R²': 0.3085649923422942,
  'Test MAE': 9.087271347086409,
  'Test MSE': 131.6058862567178},
 'XGBoost': {'Validation R²': 0.38518283037750256,
  'Validation MAE': 8.445892044750428,
  'Validation MSE': 117.5423320868714,
  'Test R²': 0.38627753368242634,
  'Test MAE': 8.43514352089679,
  'Test MSE': 116.81428941383277},
 'CatBoost': {'Validation R²': 0.3093933509363136,
  'Validation MAE': 9.099730861721103,
  'Validation MSE': 132.03196022565163,
  'Test R²': 0.3085649923422942,
  'Test MAE': 9.087271347086409,
  'Test MSE': 131.6058862567178}}

#CARVEME-GR-III-100

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.420135 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 984
[LightGBM] [Info] Number of data points in the train set: 163461, number of used features: 492
[LightGBM] [Info] Start training from score 44.225934
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:06<00:13,  6.60s/model]

Validation Report for LightGBM:
R²: 0.4885, MAE: 9.7018, MSE: 151.1449

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:00:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


...

Training Models:  67%|██████▋   | 2/3 [00:12<00:06,  6.38s/model]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: CatBoost
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.415336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 984
[LightGBM] [Info] Number of data points in the train set: 163461, number of used features: 492
[LightGBM] [Info] Start training from score 44.225934
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:19<00:00,  6.48s/model]

Validation Report for CatBoost:
R²: 0.4885, MAE: 9.7018, MSE: 151.1449

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.4885, Val MAE = 9.7018, Val MSE = 151.1449, Test R² = 0.4911, Test MAE = 9.6871, Test MSE = 150.9243
XGBoost: Val R² = 0.5317, Val MAE = 9.2628, Val MSE = 138.3743, Test R² = 0.5350, Test MAE = 9.2319, Test MSE = 137.8929
CatBoost: Val R² = 0.4885, Val MAE = 9.7018, Val MSE = 151.1449, Test R² = 0.4911, Test MAE = 9.6871, Test MSE = 150.9243

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.48846829455062846,
  'Validation MAE': 9.70181198804441,
  'Validation MSE': 151.14493906919625,
  'Test R²': 0.4910558157194159,
  'Test MAE': 9.687108974695544,
  'Test MSE': 150.92428909888088},
 'XGBoost': {'Validation R²': 0.5316888105348908,
  'Validation MAE': 9.26282677678163,
  'Validation MSE': 138.3743479496452,
  'Test R²': 0.5350000399670904,
  'Test MAE': 9.231885698571798,
  'Test MSE': 137.892897819782},
 'CatBoost': {'Validation R²': 0.48846829455062846,
  'Validation MAE': 9.70181198804441,
  'Validation MSE': 151.14493906919625,
  'Test R²': 0.4910558157194159,
  'Test MAE': 9.687108974695544,
  'Test MSE': 150.92428909888088}}

#CARVEME-GR-I-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.125511 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 968
[LightGBM] [Info] Number of data points in the train set: 56325, number of used features: 484
[LightGBM] [Info] Start training from score 36.686596
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:01<00:03,  1.81s/model]

Validation Report for LightGBM:
R²: 0.2809, MAE: 9.2152, MSE: 134.4994

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:03:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:06<00:03,  3.56s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.096043 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 968
[LightGBM] [Info] Number of data points in the train set: 56325, number of used features: 484
[LightGBM] [Info] Start training from score 36.686596


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:08<00:00,  2.73s/model]

Model saved at results/CatBoost/model.pkl
Validation Report for CatBoost:
R²: 0.2809, MAE: 9.2152, MSE: 134.4994

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.2809, Val MAE = 9.2152, Val MSE = 134.4994, Test R² = 0.2712, Test MAE = 9.2152, Test MSE = 134.6943
XGBoost: Val R² = 0.2810, Val MAE = 9.1593, Val MSE = 134.4828, Test R² = 0.2674, Test MAE = 9.1736, Test MSE = 135.3948
CatBoost: Val R² = 0.2809, Val MAE = 9.2152, Val MSE = 134.4994, Test R² = 0.2712, Test MAE = 9.2152, Test MSE = 134.6943

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.2809237383712012,
  'Validation MAE': 9.215203857901106,
  'Validation MSE': 134.49936314084414,
  'Test R²': 0.2712338785983074,
  'Test MAE': 9.215225566109597,
  'Test MSE': 134.69432616081144},
 'XGBoost': {'Validation R²': 0.28101230182158143,
  'Validation MAE': 9.159346563916252,
  'Validation MSE': 134.48279782182402,
  'Test R²': 0.26744390022257325,
  'Test MAE': 9.173603511708592,
  'Test MSE': 135.39480957859394},
 'CatBoost': {'Validation R²': 0.2809237383712012,
  'Validation MAE': 9.215203857901106,
  'Validation MSE': 134.49936314084414,
  'Test R²': 0.2712338785983074,
  'Test MAE': 9.215225566109597,
  'Test MSE': 134.69432616081144}}

#CARVEME-GR-II-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.091164 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 968
[LightGBM] [Info] Number of data points in the train set: 56325, number of used features: 484
[LightGBM] [Info] Start training from score 36.685596
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:01<00:02,  1.49s/model]

Validation Report for LightGBM:
R²: 0.2809, MAE: 9.2152, MSE: 134.4994

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:04:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
Training Models:  67%|██████▋   | 2/3 [00:03<00:01,  1.80s/model]

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.134069 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 968
[LightGBM] [Info] Number of data points in the train set: 56325, number of used features: 484
[LightGBM] [Info] Start training from score 36.685596
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:05<00:00,  1.81s/model]

Validation Report for CatBoost:
R²: 0.2809, MAE: 9.2152, MSE: 134.4994

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.2809, Val MAE = 9.2152, Val MSE = 134.4994, Test R² = 0.2712, Test MAE = 9.2152, Test MSE = 134.6943
XGBoost: Val R² = 0.2810, Val MAE = 9.1593, Val MSE = 134.4828, Test R² = 0.2674, Test MAE = 9.1736, Test MSE = 135.3948
CatBoost: Val R² = 0.2809, Val MAE = 9.2152, Val MSE = 134.4994, Test R² = 0.2712, Test MAE = 9.2152, Test MSE = 134.6943

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.2809237384127711,
  'Validation MAE': 9.215203857746685,
  'Validation MSE': 134.49936313306875,
  'Test R²': 0.2712338786667591,
  'Test MAE': 9.215225565790709,
  'Test MSE': 134.69432614815986},
 'XGBoost': {'Validation R²': 0.28101229645489256,
  'Validation MAE': 9.159346585179406,
  'Validation MSE': 134.48279882563443,
  'Test R²': 0.26744389657860834,
  'Test MAE': 9.173603546325522,
  'Test MSE': 135.3948102520904},
 'CatBoost': {'Validation R²': 0.2809237384127711,
  'Validation MAE': 9.215203857746685,
  'Validation MSE': 134.49936313306875,
  'Test R²': 0.2712338786667591,
  'Test MAE': 9.215225565790709,
  'Test MSE': 134.69432614815986}}


#CARVEME-GR-III-50

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = load(train_list)

models = {
    "LightGBM": LGBMRegressor(random_state=42),
    "XGBoost": XGBRegressor(use_label_encoder=False, random_state=42),
    "CatBoost": LGBMRegressor(random_state=42),
}

train_val_and_test(models, X_train, y_train, X_val, y_val, X_test, y_test, seed=4221)

Training Models:   0%|          | 0/3 [00:00<?, ?model/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Training and evaluating: LightGBM
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.400467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 982
[LightGBM] [Info] Number of data points in the train set: 235662, number of used features: 491
[LightGBM] [Info] Start training from score 36.427294
Model saved at results/LightGBM/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models:  33%|███▎      | 1/3 [00:06<00:12,  6.42s/model]

Validation Report for LightGBM:
R²: 0.4444, MAE: 9.6070, MSE: 149.0280

Training and evaluating: XGBoost


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:06:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.383140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 982
[LightGBM] [Info] Number of data points in the train set: 235662, number of used features: 491
[LightGBM] [Info] Start training from score 36.427294
Model saved at results/CatBoost/model.pkl


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
Training Models: 100%|██████████| 3/3 [00:21<00:00,  7.33s/model]

Validation Report for CatBoost:
R²: 0.4444, MAE: 9.6070, MSE: 149.0280

Summary of Validation and Test Metrics:
LightGBM: Val R² = 0.4444, Val MAE = 9.6070, Val MSE = 149.0280, Test R² = 0.4405, Test MAE = 9.6390, Test MSE = 150.2734
XGBoost: Val R² = 0.4570, Val MAE = 9.5213, Val MSE = 145.6456, Test R² = 0.4563, Test MAE = 9.5388, Test MSE = 146.0393
CatBoost: Val R² = 0.4444, Val MAE = 9.6070, Val MSE = 149.0280, Test R² = 0.4405, Test MAE = 9.6390, Test MSE = 150.2734

All model results saved to results/all_model_results.json


{'LightGBM': {'Validation R²': 0.4443902775440487,
  'Validation MAE': 9.607017525801249,
  'Validation MSE': 149.02798695673155,
  'Test R²': 0.44053593834293425,
  'Test MAE': 9.638958504744306,
  'Test MSE': 150.27340886653337},
 'XGBoost': {'Validation R²': 0.45700052576751404,
  'Validation MAE': 9.521263664851475,
  'Validation MSE': 145.64561290564262,
  'Test R²': 0.45629945714956355,
  'Test MAE': 9.538802471621551,
  'Test MSE': 146.0392893418803},
 'CatBoost': {'Validation R²': 0.4443902775440487,
  'Validation MAE': 9.607017525801249,
  'Validation MSE': 149.02798695673155,
  'Test R²': 0.44053593834293425,
  'Test MAE': 9.638958504744306,
  'Test MSE': 150.27340886653337}}